In [1]:
import matplotlib.pyplot as plt
from collections import defaultdict
# from wordcloud import WordCloud
import pandas as pd
import plotly.figure_factory as ff
# from src.model import apriori
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import plotly.plotly as py 

In [2]:
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt

In [4]:
import numpy as np
from collections import defaultdict
import itertools

In [5]:
import sys
project_path = "/home/ashmi/Amazon-Mining"
# make sure to use position 1
sys.path.insert(1, project_path)

In [6]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [7]:
def get_cat(l):
    return l[-1]

## data loading

In [8]:
from src.data.json_loader import JSONLoader
# file_path corresponds to the file of the .gz file which contains the JSON file. 
product_path = '/home/ashmi/Desktop/Amazondata/meta_Electronics.json.gz'
loader = JSONLoader()
product = loader.load_data(product_path)

In [9]:
reviews_path = '/home/ashmi/Desktop/Amazondata/reviews_Electronics_5.json.gz'
reviews = loader.load_data(reviews_path)

In [10]:
product.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [11]:
product.categories = product.categories.apply(flatten)

In [12]:
product.shape

(498196, 9)

In [13]:
product.categories

0         [Electronics, Computers & Accessories, Cables ...
1         [Electronics, Computers & Accessories, Cables ...
2         [Electronics, Computers & Accessories, PDAs, H...
3         [Electronics, Accessories & Supplies, Audio & ...
4         [Electronics, GPS & Navigation, Vehicle GPS, T...
5         [Electronics, Accessories & Supplies, Audio & ...
6         [Electronics, eBook Readers & Accessories, Pow...
7         [Electronics, eBook Readers & Accessories, Skins]
8         [Electronics, eBook Readers & Accessories, Cov...
9         [Electronics, eBook Readers & Accessories, Cov...
10        [Electronics, eBook Readers & Accessories, Cov...
11        [Electronics, eBook Readers & Accessories, Cov...
12        [Electronics, Computers & Accessories, Touch S...
13        [Electronics, eBook Readers & Accessories, Cov...
14        [Electronics, Accessories & Supplies, Audio & ...
15        [Electronics, Computers & Accessories, Touch S...
16        [Electronics, eBook Readers & 

In [14]:
product.ix[0]

/home/ashmi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning:


.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated



asin                                                  0132793040
imUrl          http://ecx.images-amazon.com/images/I/31JIPhp%...
description    The Kelby Training DVD Mastering Blend Modes i...
categories     [Electronics, Computers & Accessories, Cables ...
title          Kelby Training DVD: Mastering Blend Modes in A...
price                                                        NaN
salesRank                                                    NaN
related                                                      NaN
brand                                                        NaN
Name: 0, dtype: object

In [15]:
reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013"
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010"
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010"
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010"
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011"


In [16]:
len(reviews.reviewerID.unique())

192403

In [17]:
reviews.shape

(1689188, 9)

# common products between 2 datasets

In [18]:
common_products = product.loc[product.asin.isin(reviews.asin), ]

In [19]:
common_products.shape

(63001, 9)

In [20]:
common_products.to_csv('common_products.csv', index=False)

In [21]:
reviews_indexed_by_reviewerID = reviews.set_index(['reviewerID'])

In [22]:
unique_reviewer = np.unique(reviews.reviewerID)

In [23]:
len(unique_reviewer)

192403

In [24]:
reviews_by_reviewer = reviews.groupby(by=['reviewerID'])['asin'].count()

In [25]:
review_to_asin = reviews.groupby(by=['reviewerID'])['asin'].unique()

In [26]:
review_to_asin

reviewerID
A000715434M800HLCENK9    [B000UYYZ0M, B001EHAI6Y, B003AFONFU, B003ES5ZU...
A00101847G3FJTWYGNQA     [B005F778JO, B006T9B6R2, B009NB8WR0, B00B19L8L...
A00166281YWM98A3SVD55    [B005HGBF9W, B005HSG3BA, B0074BW614, B007B5S8B...
A0046696382DWIPVIWO0K    [B0025ZUF8K, B002TTKC2G, B00CH643A8, B00E055H5...
A00472881KT6WR48K907X    [B0000AZJZT, B003LTRLPY, B0053OLY9O, B007UIX1M...
A00473363TJ8YSZ3YAGG9    [B002LB42NM, B0039BYJ4A, B003MX4D0U, B003NR57B...
A005721627VX5W2COKKK2    [B001T9N0SU, B002WAZ0B0, B0031MGUBC, B003ELYQG...
A00700212KB3K0MVESPIY    [B000IXNEI4, B000VWPULQ, B003WUBIZQ, B0041Q38N...
A007780739H92ZWKGVWGJ    [B000S5Q9CA, B000VOE466, B004LVMCMS, B007BYWJU...
A00814373OZEDXYMXP04T    [B000P910OG, B0015V1H28, B002A6CWCQ, B002HK0UQ...
A01036691ZFOFCXBLP2D1    [B00066IJPQ, B000BUIP6K, B001A4HAFS, B005HPSFW...
A0103849GBVWICKXD4T6     [B000067RRX, B003MG9F78, B003ZSP0WW, B009A6CZY...
A0104775CQOXJTV4OH68     [B001TIG36C, B002V88HFE, B004Y09I9Q, B008DC9E8...
A0110255Y4ER0R

# merging 2 datasets

In [27]:
merged_reviews = reviews.merge(product, on='asin')

In [28]:
merged_reviews.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,imUrl,description,categories,title,price,salesRank,related,brand
0,AO94DHGC771SJ,0528881469,amazdnu,"[0, 0]",We got this GPS for my husband who is an (OTR)...,5.0,Gotta have GPS!,1370131200,"06 2, 2013",http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[Electronics, GPS & Navigation, Vehicle GPS, T...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
1,AMO214LNFCEI4,0528881469,Amazon Customer,"[12, 15]","I'm a professional OTR truck driver, and I bou...",1.0,Very Disappointed,1290643200,"11 25, 2010",http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[Electronics, GPS & Navigation, Vehicle GPS, T...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
2,A3N7T0DY83Y4IG,0528881469,C. A. Freeman,"[43, 45]","Well, what can I say. I've had this unit in m...",3.0,1st impression,1283990400,"09 9, 2010",http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[Electronics, GPS & Navigation, Vehicle GPS, T...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
3,A1H8PY3QHMQQA0,0528881469,"Dave M. Shaw ""mack dave""","[9, 10]","Not going to write a long review, even thought...",2.0,"Great grafics, POOR GPS",1290556800,"11 24, 2010",http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[Electronics, GPS & Navigation, Vehicle GPS, T...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN
4,A24EV6RXELQZ63,0528881469,Wayne Smith,"[0, 0]",I've had mine for a year and here's what we go...,1.0,"Major issues, only excuses for support",1317254400,"09 29, 2011",http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[Electronics, GPS & Navigation, Vehicle GPS, T...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [29]:
merged_categories_overall_brand_data = merged_reviews[['reviewerID','asin','overall','categories','brand']].copy()

In [30]:
merged_categories_overall_brand_data.head()

,reviewerID,asin,overall,categories,brand
0,AO94DHGC771SJ,0528881469,5.0,"[Electronics, GPS & Navigation, Vehicle GPS, T...",NaN
1,AMO214LNFCEI4,0528881469,1.0,"[Electronics, GPS & Navigation, Vehicle GPS, T...",NaN
2,A3N7T0DY83Y4IG,0528881469,3.0,"[Electronics, GPS & Navigation, Vehicle GPS, T...",NaN
3,A1H8PY3QHMQQA0,0528881469,2.0,"[Electronics, GPS & Navigation, Vehicle GPS, T...",NaN
4,A24EV6RXELQZ63,0528881469,1.0,"[Electronics, GPS & Navigation, Vehicle GPS, T...",NaN


In [31]:
merged_categories_overall_brand_data.dropna(axis=0, subset=['brand'], inplace=True) # dropping NaNs from brand


In [32]:
merged_categories_overall_brand_data.head()

,reviewerID,asin,overall,categories,brand
10,A7S2B0I67WNWB,0594481813,4.0,"[Electronics, eBook Readers & Accessories, Pow...",Barnes &amp; Noble
11,A3HICVLF4PFFMN,0594481813,5.0,"[Electronics, eBook Readers & Accessories, Pow...",Barnes &amp; Noble
12,ANSKSPEEAKY7S,0594481813,5.0,"[Electronics, eBook Readers & Accessories, Pow...",Barnes &amp; Noble
13,A2QBZA4S1ROX9Q,0594481813,3.0,"[Electronics, eBook Readers & Accessories, Pow...",Barnes &amp; Noble
14,ANY6JUFM0GH8U,0594481813,5.0,"[Electronics, eBook Readers & Accessories, Pow...",Barnes &amp; Noble


In [33]:
merged_categories_overall_brand_data.categories = merged_categories_overall_brand_data.categories.apply(get_cat) # getting the lowest level categories

In [35]:
merged_categories_overall_brand_data.head()

,reviewerID,asin,overall,categories,brand
10,A7S2B0I67WNWB,0594481813,4.0,Power Adapters,Barnes &amp; Noble
11,A3HICVLF4PFFMN,0594481813,5.0,Power Adapters,Barnes &amp; Noble
12,ANSKSPEEAKY7S,0594481813,5.0,Power Adapters,Barnes &amp; Noble
13,A2QBZA4S1ROX9Q,0594481813,3.0,Power Adapters,Barnes &amp; Noble
14,ANY6JUFM0GH8U,0594481813,5.0,Power Adapters,Barnes &amp; Noble


In [36]:
len(merged_categories_overall_brand_data)

954251

In [39]:
positive_data = merged_categories_overall_brand_data.loc[merged_categories_overall_brand_data['overall']> 3.0]


In [40]:
negative_data = merged_categories_overall_brand_data.loc[merged_categories_overall_brand_data['overall']< 3.0]


In [41]:
positive_data.head()

,reviewerID,asin,overall,categories,brand
10,A7S2B0I67WNWB,0594481813,4.0,Power Adapters,Barnes &amp; Noble
11,A3HICVLF4PFFMN,0594481813,5.0,Power Adapters,Barnes &amp; Noble
12,ANSKSPEEAKY7S,0594481813,5.0,Power Adapters,Barnes &amp; Noble
14,ANY6JUFM0GH8U,0594481813,5.0,Power Adapters,Barnes &amp; Noble
17,A1S6B5QFWGVL5U,0594481813,4.0,Power Adapters,Barnes &amp; Noble


In [42]:
len(positive_data)

771070

In [43]:
len(negative_data)

104502

In [44]:
negative_data.head()

,reviewerID,asin,overall,categories,brand
68,A2LR9WP2JGDT8E,0972683275,2.0,TV Ceiling & Wall Mounts,VideoSecu
71,A2JMN2JA9LSHVL,0972683275,1.0,TV Ceiling & Wall Mounts,VideoSecu
82,A38FGQVJM18OWV,0972683275,1.0,TV Ceiling & Wall Mounts,VideoSecu
167,A15K7HV1XD6YWR,0972683275,1.0,TV Ceiling & Wall Mounts,VideoSecu
201,A11ZP93TJED4T6,0972683275,2.0,TV Ceiling & Wall Mounts,VideoSecu


In [78]:
positive_data_gb = positive_data.groupby(['brand'])

In [122]:
negative_data_gb = negative_data.groupby(['brand'])

In [79]:
positive_data_gb.head()

,reviewerID,asin,overall,categories,brand
10,A7S2B0I67WNWB,0594481813,4.0,Power Adapters,Barnes &amp; Noble
11,A3HICVLF4PFFMN,0594481813,5.0,Power Adapters,Barnes &amp; Noble
12,ANSKSPEEAKY7S,0594481813,5.0,Power Adapters,Barnes &amp; Noble
14,ANY6JUFM0GH8U,0594481813,5.0,Power Adapters,Barnes &amp; Noble
17,A1S6B5QFWGVL5U,0594481813,4.0,Power Adapters,Barnes &amp; Noble
18,A20XXTXWF2TCPY,0972683275,5.0,TV Ceiling & Wall Mounts,VideoSecu
19,A2IDCSC6NVONIZ,0972683275,5.0,TV Ceiling & Wall Mounts,VideoSecu
20,A1EDI0X3GI1SK7,0972683275,5.0,TV Ceiling & Wall Mounts,VideoSecu
21,A3BMUBUC1N77U8,0972683275,4.0,TV Ceiling & Wall Mounts,VideoSecu
22,AVRFGGCCCR6QU,0972683275,4.0,TV Ceiling & Wall Mounts,VideoSecu


In [81]:
positive_brand_df = pd.DataFrame(positive_data_gb.size().reset_index(name = "brand_review_popularity"))


In [123]:
negative_brand_df = pd.DataFrame(negative_data_gb.size().reset_index(name = "brand_review_popularity"))

In [83]:
positive_brand_df

,brand,brand_review_popularity
0,,1578
1,(CHARGER4U),4
2,0722301698334,4
3,12vAdapters.com,5
4,1800Battery,23
5,180s,4
6,1d4,5
7,1stbuyer,8
8,2791 2795,6
9,2K,59


In [117]:
sorted_brand_positive = positive_brand_df.sort_values(by='brand_review_popularity')


In [127]:
sorted_brand_negative = negative_brand_df.sort_values(by='brand_review_popularity')


In [128]:
sorted_brand_negative = sorted_brand_negative[-10:]

In [129]:
sorted_brand_negative

,brand,brand_review_popularity
1005,Garmin,1303
2769,eForCity,1393
2625,Western Digital,1455
210,Asus,1612
1681,Netgear,1659
2107,SanDisk,1791
257,BELKIN,1831
2226,Sony,2183
1019,Generic,3007
1451,Logitech,3517


In [118]:
sorted_brand_positive = sorted_brand_positive[-5:]

In [119]:
sorted_brand_positive.brand

306       BELKIN
2716        Sony
572        Canon
2572     SanDisk
1771    Logitech
Name: brand, dtype: object

In [130]:
sorted_brand_positive_list = list(sorted_brand_positive.brand)
sorted_brand_negative_list = list(sorted_brand_negative.brand)

In [121]:
top_brand_positive_data = (positive_data.loc[positive_data['brand'].isin((sorted_brand_positive_list))]) # only extracting data relevant to the top 10 brands by popularity


In [131]:
top_brand_negative_data = (negative_data.loc[negative_data['brand'].isin((sorted_brand_negative_list))]) # only extracting data relevant to the top 10 brands by popularity


In [132]:
top_brand_negative_data.head()

,reviewerID,asin,overall,categories,brand
835,AB7DR7FWLMGZ9,9862510447,1.0,PC Microphones,Generic
839,A2A28DJDS6LEHP,9876050621,1.0,Micro SD Cards,SanDisk
840,A2I9X30JGIAG,9876050621,1.0,Micro SD Cards,SanDisk
1003,A2A3BLPBHISRZ4,9984984354,1.0,Chargers & Cables,Garmin
1009,A2WXLG4AYEDOQI,9984984354,1.0,Chargers & Cables,Garmin


In [94]:
top_brand_positive_data.head()

,reviewerID,asin,overall,categories,brand
640,A2GKUSRRPCOV17,3744295508,5.0,HDMI Cables,Generic
641,A3F2M27L02YDLM,3744295508,5.0,HDMI Cables,Generic
642,A2D23Q75DN4LLH,3744295508,5.0,HDMI Cables,Generic
643,A2N3OJ1T4IKZBM,3744295508,5.0,HDMI Cables,Generic
644,A3ORP2IZ7D19VP,3744295508,5.0,HDMI Cables,Generic


In [101]:
top_brand_positive = top_brand_positive_data.groupby(['brand','categories']).size().reset_index(name="categories_count")

In [133]:
top_brand_negative = top_brand_negative_data.groupby(['brand','categories']).size().reset_index(name="categories_count")

In [102]:
top_brand_positive

,brand,categories,categories_count
0,Asus,AC Adapters,22
1,Asus,Amps,20
2,Asus,Batteries,3
3,Asus,Blu-ray Drives,137
4,Asus,Bundles,18
5,Asus,Cables & Interconnects,17
6,Asus,Cases,124
7,Asus,Chargers & Adapters,110
8,Asus,Computer Cable Adapters,2
9,Asus,Computer Components,57


In [134]:
top_brand_negative

,brand,categories,categories_count
0,Asus,AC Adapters,1
1,Asus,Amps,3
2,Asus,Batteries,2
3,Asus,Blu-ray Drives,22
4,Asus,Bundles,7
5,Asus,Cables & Interconnects,1
6,Asus,Cases,71
7,Asus,Chargers & Adapters,22
8,Asus,Computer Components,7
9,Asus,DVD Drives,2


In [103]:
top_brand_positive_gb_brand = top_brand_positive.groupby(['brand'])

In [135]:
top_brand_negative_gb_brand = top_brand_negative.groupby(['brand'])

In [136]:
top_brand_negative_gb_brand.head()

,brand,categories,categories_count
0,Asus,AC Adapters,1
1,Asus,Amps,3
2,Asus,Batteries,2
3,Asus,Blu-ray Drives,22
4,Asus,Bundles,7
41,BELKIN,Accessories & Supplies,2
42,BELKIN,Adapters,46
43,BELKIN,Antennas,105
44,BELKIN,Armbands,44
45,BELKIN,Audio Cables,8


In [104]:
top_brand_positive_gb_brand.head()

,brand,categories,categories_count
0,Asus,AC Adapters,22
1,Asus,Amps,20
2,Asus,Batteries,3
3,Asus,Blu-ray Drives,137
4,Asus,Bundles,18
45,BELKIN,Accessories & Supplies,29
46,BELKIN,Adapters,473
47,BELKIN,Antennas,291
48,BELKIN,Armbands,135
49,BELKIN,Audio Cables,135


In [105]:
import plotly.plotly as py
import plotly.graph_objs as go



In [140]:
def plot_graph(brand, cat_list, count_list):
#     print("brand",brand)
#     print("cat_list",cat_list)
#     print("count_list",count_list)
    fig = {
        "data": [go.Bar(
                x=cat_list,
                y=count_list
        )]}
    
    save_file_name = brand + 'negative'+'basic_bar'
    py.plot(fig, filename= save_file_name)

In [139]:
 for (idx,br) in enumerate(top_brand_positive_gb_brand):
    print("Br",br[0])
    for (cat,count) in zip(top_brand_positive_gb_brand.categories,top_brand_positive_gb_brand.categories_count):
        cat_list = list(cat)
#         print("cat_list[1]",cat_list[1])
#         print("cat_list[0]",cat_list[0]) # brand name
        count_list = list(count)
#         print("count_list[1]",count_list[1])
        plot_graph(cat_list[0],cat_list[1],count_list[1])

Br Asus


KeyboardInterrupt: 

In [142]:
 for (idx,br) in enumerate(top_brand_negative_gb_brand):
    print("Br",br[0])
    for (cat,count) in zip(top_brand_negative_gb_brand.categories,top_brand_negative_gb_brand.categories_count):
        cat_list = list(cat)
#         print("cat_list[1]",cat_list[1])
#         print("cat_list[0]",cat_list[0]) # brand name
        count_list = list(count)
#         print("count_list[1]",count_list[1])
        plot_graph(cat_list[0],cat_list[1],count_list[1])

Br Asus
Br BELKIN
Br Garmin
Br Generic


PlotlyRequestError: Hi there, you've reached the threshold of 100 combined image exports and chart saves per day. If you need to raise your daily limit, consider upgrading to a Student or Personal Plan (see: https://plot.ly/products/cloud).